In [1]:
import numpy as np
from typing import List
import tensorflow as tf
import pisces.models as pm
from importlib import reload
from matplotlib import pyplot as plt
from sklearn.model_selection import LeaveOneOut
from pisces.data_sets import DataSetObject, ModelInputSpectrogram, ModelOutputType, DataProcessor, PSGType

## Set up paths for caching
Set up all the paths here, ensure the (sub)folders all exist so later when we try to save to these there are no errors.

In [2]:

import os
from pathlib import Path

CWD = Path(os.getcwd())
save_path = CWD.joinpath("pre_processed_data")
hybrid_path = save_path.joinpath("hybrid")
os.makedirs(hybrid_path, exist_ok=True)
disordered_path = save_path.joinpath("disordered")
os.makedirs(disordered_path, exist_ok=True)
walch_path = save_path.joinpath("walch")
os.makedirs(walch_path, exist_ok=True)
data_location = Path('/Users/eric/Engineering/Work/pisces/data_sets')#CWD.parent.joinpath("data_sets")
# data_location = Path('/home/eric/Engineering/Work/pisces/data_sets')#CWD.parent.joinpath("data_sets")


In [3]:
data_location

PosixPath('/Users/eric/Engineering/Work/pisces/data_sets')

In [4]:
!pisces_setup

Converting Mads Olsen model to Keras...
Model saved at /Users/eric/Engineering/Work/pisces/pisces/cached_models/mo_resunet.keras


# Walch

In [5]:
sets = DataSetObject.find_data_sets(data_location)
walch = sets['walch_et_al']
walch.parse_data_sets()
print(f"Found {len(walch.ids)} subjects")

hybrid = sets['hybrid_motion']
hybrid.parse_data_sets()
print(f"Found {len(hybrid.ids)} subjects")

Found 31 subjects
Found 31 subjects


In [6]:
subjects_to_exclude_walch = [
    "3509524", "5383425",
    "7749105","759667",
    "8258170",
    "9961348", "5132496",
]

# SAME subjects, hybrid is fuzzed data
subjects_to_exclude_hybrid = subjects_to_exclude_walch

In [7]:
sampling_hz = 32 # Hz
input_features = ['accelerometer']
model_input = ModelInputSpectrogram(input_features, sampling_hz)
output_type = ModelOutputType.WAKE_LIGHT_DEEP_REM
data_processor_walch = DataProcessor(walch, model_input, output_type=output_type,
                                     psg_type=PSGType.HAS_N4)

In [8]:
mo = pm.MOResUNetPretrained(data_processor=data_processor_walch, sampling_hz=sampling_hz)

In [9]:
import sys
from typing import Dict
import numpy as np
from scipy.signal import spectrogram

import pisces.data_sets as pds

FIXED_LABEL_LENGTH = 1024
# FIXED_SPECGRAM_SHAPE = (15360, 32, 3)
FIXED_SPECGRAM_SHAPE = (15360, 32)

ACC_RAW_HZ = 50
ACC_RAW_DT = 1/ACC_RAW_HZ
ACC_INPUT_HZ = 32
ACTIVITY_RATE = 15
ACTIVITY_HZ = 1/ACTIVITY_RATE
PSG_RATE = 30
PSG_HZ = 1/PSG_RATE
SECONDS_PER_KERNEL = 5 * 60
ACTIVITY_KERNEL_WIDTH = SECONDS_PER_KERNEL * ACTIVITY_HZ
ACTIVITY_KERNEL_WIDTH += 1 - (ACTIVITY_KERNEL_WIDTH % 2)  # Ensure it is odd

def process_data(dataset: pds.DataSetObject, id, ACTIVITY_KERNEL_WIDTH):
    # Load data
    accel_data = dataset.get_feature_data('accelerometer', id).to_numpy()
    activity_data = dataset.get_feature_data('activity', id).to_numpy()
    psg_data = dataset.get_feature_data('psg', id).to_numpy()

    # Sort based on time (axis 0)
    accel_data = accel_data[accel_data[:, 0].argsort()]
    activity_data = activity_data[activity_data[:, 0].argsort()]
    psg_data = psg_data[psg_data[:, 0].argsort()]

    # Convert activity and PSG time to int
    activity_data[:, 0] = np.round(activity_data[:, 0])
    psg_data[:, 0] = np.round(psg_data[:, 0])
    
    # Trim data to common time range
    start_time = max(accel_data[0, 0], activity_data[0, 0], psg_data[0, 0])
    end_time = min(accel_data[-1, 0], activity_data[-1, 0], psg_data[-1, 0])
    
    accel_data = accel_data[(accel_data[:, 0] >= start_time) & (accel_data[:, 0] <= end_time)]
    activity_data = activity_data[(activity_data[:, 0] >= start_time) & (activity_data[:, 0] <= end_time)]
    psg_data = psg_data[(psg_data[:, 0] >= start_time) & (psg_data[:, 0] <= end_time)]

    # print("PSG Shape", psg_data.shape)
    
    # Find gaps in accelerometer data
    time_diff = np.diff(accel_data[:, 0])
    avg_time_hz = int(1/np.median(time_diff))
    avg_time_diff = 1/avg_time_hz
    threshold = 100 * avg_time_diff
    gap_indices = np.where(time_diff > threshold)[0]
    # print("avg_time_diff", avg_time_diff)
    # print("threshold", threshold)
    # print(f"Found {len(gap_indices)} gaps in accelerometer data")
    
    # Mask PSG labels during accelerometer gaps
    n_psg_mask = np.sum(psg_data[:, 1] < 0)
    len_psg = psg_data.shape[0]
    # print("# of masked epochs: ", n_psg_mask, "out of", len_psg, f"({100 * n_psg_mask / len_psg:.2f}%)")
    pre_mask_sleeps = np.sum(psg_data[:, 1] > 0)
    pre_mask_wakes = np.sum(psg_data[:, 1] == 0)
    wakes_masked = 0
    print("Pre-mask:\n\tSleeps", pre_mask_sleeps, "\n\tWakes", pre_mask_wakes)
    for gap_index in gap_indices:
        gap_start = accel_data[gap_index, 0] + avg_time_diff
        gap_end = accel_data[gap_index + 1, 0]
        mask_indices = np.where((psg_data[:, 0] + PSG_RATE >= gap_start) & (psg_data[:, 0] <= gap_end))[0]
        wake_counts = np.sum((psg_data[mask_indices, 1].astype(int)) == 0)
        wakes_masked += wake_counts
        sleep_counts = np.sum((psg_data[mask_indices, 1].astype(int)) > 0)
        # print(f"Gap {gap_index}:\n\twakes masked: {wake_counts}\n\tsleeps masked: {sleep_counts}")
        psg_data[mask_indices, 1:] = -1  # Assuming labels are in columns from index 1 onwards
    # print("END gap masking: # of masked epochs: ", np.sum(psg_data[:, 1] < 0), "out of", len_psg)
    
    # Mask PSG labels at start and end based on ACTIVITY_KERNEL_WIDTH
    labels_per_activity = PSG_RATE * ACTIVITY_HZ
    num_labels_to_mask = int(np.ceil((ACTIVITY_KERNEL_WIDTH // 2) // labels_per_activity))
    if num_labels_to_mask > 0:
        # print(f"MASKING {num_labels_to_mask} LABELS from either end")
        psg_data[:num_labels_to_mask, 1:] = -1
        psg_data[-num_labels_to_mask:, 1:] = -1
    post_mask_sleeps = np.sum(psg_data[:, 1] > 0)
    post_mask_wakes = np.sum(psg_data[:, 1] == 0)
    print("Post-mask:\n\tSleeps", post_mask_sleeps, "\n\tWakes", post_mask_wakes, "\n\tWakes masked", wakes_masked)
    
    # Convert accelerometer data to spectrograms
    accel_data_resampled = resample_accel_data(accel_data, original_fs=int(avg_time_hz), target_fs=ACC_INPUT_HZ)
    spectrograms = data_processor_walch.accelerometer_to_spectrogram(accel_data_resampled)
    # spectrograms = data_processor_walch.accelerometer_to_spectrogram(accel_data)
    # spectrograms = convert_accel_to_spectrogram(accel_data_resampled)
    padded_spectrograms = np.zeros(FIXED_SPECGRAM_SHAPE)
    padded_spectrograms[:spectrograms.shape[0], ...] = spectrograms[:FIXED_SPECGRAM_SHAPE[0], ...]
    
    # Pad PSG data to 1024 samples
    psg_data = pad_or_truncate(psg_data, 1024)
    
    # Pad activity data to 2 * 1024 samples
    activity_data = pad_or_truncate(activity_data, int(labels_per_activity * FIXED_LABEL_LENGTH))
    
    return {"spectrogram": padded_spectrograms, "activity": activity_data, "psg": psg_data}

import numpy as np
from scipy.signal import resample_poly

def resample_accel_data(accel_data, original_fs, target_fs=ACC_INPUT_HZ):
    # Calculate the greatest common divisor for up/down sampling rates
    up = target_fs
    down = original_fs
    gcd = np.gcd(int(up), int(down))
    up = int(up // gcd)
    down = int(down // gcd)

    accel_resampled_time = np.arange(accel_data[0, 0], accel_data[-1, 0], 1/original_fs)

    accel_resampled = np.zeros((len(accel_resampled_time), accel_data.shape[1]))
    accel_resampled[:, 0] = accel_resampled_time

    for i in range(1, accel_data.shape[1]):
        accel_resampled[:, i] = np.interp(accel_resampled_time, accel_data[:, 0], accel_data[:, i])
    
    # Resample data (excluding the time column)
    resampled_data = resample_poly(accel_resampled[:, 1:], up, down, axis=0)
    
    # Recompute the time vector
    duration = accel_data[-1, 0] - accel_data[0, 0]
    num_samples = resampled_data.shape[0]
    new_time = np.linspace(accel_resampled_time[0], accel_resampled_time[-1], num_samples)
    
    # Combine the new time vector with the resampled data
    resampled_accel_data = np.column_stack((new_time, resampled_data))
    return resampled_accel_data

def cal_psd(x, fs=ACC_INPUT_HZ, window=320, noverlap=256, nfft=512, f_min=0, f_max=6, f_sub=3):
    """
    https://github.com/MADSOLSEN/SleepStagePrediction/blob/d47ff488f5cedd3b0459593a53fc4f92fc3660a2/signal_processing/spectrogram.py#L91
    """
    from scipy.ndimage import maximum_filter as maxfilt

    # border edit
    x_ = np.zeros((x.size + window,))
    x_[window // 2 : -window // 2] = x
    x_ = x_ + np.random.normal(loc=0, scale=sys.float_info.epsilon, size=(x_.shape))

    f, t, S = spectrogram(
        x=x_,
        fs=fs,
        window=np.blackman(window),
        nperseg=window,
        noverlap=noverlap,
        nfft=nfft,
    )

    S = S[(f > f_min) & (f <= f_max), :]
    S = maxfilt(np.abs(S), size=(f_sub, 1))
    S = S[::f_sub, :]
    S = np.swapaxes(S, axis1=1, axis2=0)
    S = np.log(S + sys.float_info.epsilon)

    return S

def convert_accel_to_spectrogram(accel_data, sampling_rate_Hz=32, nfft=512, window=320, noverlap=256):
    spectrograms = []
    for axis in range(1, accel_data.shape[1]):  # Skip time column
        Sxx = cal_psd(accel_data[:, axis], fs=sampling_rate_Hz, nfft=nfft, window=window, noverlap=noverlap)
        spectrograms.append(Sxx)
    spectrograms = np.stack(spectrograms, axis=-1)
    return spectrograms

def pad_or_truncate(data, desired_length, pad_value: float = -1.0):
    current_length = data.shape[0]
    if current_length < desired_length:
        padding = desired_length - current_length
        pad_width = ((0, padding), (0, 0))
        data = np.pad(data, pad_width, mode='constant', constant_values=pad_value)
    else:
        data = data[:desired_length, :]
    return data


In [10]:
def process_data_set(data_set: pds.DataSetObject, ids_to_exclude: List[str], ACTIVITY_KERNEL_WIDTH):
    data = {}
    for id in data_set.ids:
        if id in ids_to_exclude:
            continue
        print(f"Processing {id}")
        data[id] = process_data(data_set, id, ACTIVITY_KERNEL_WIDTH)
    return data

In [11]:
preprocessed_data_walch = process_data_set(walch, subjects_to_exclude_walch, ACTIVITY_KERNEL_WIDTH)
preprocessed_data_hybrid = process_data_set(hybrid, subjects_to_exclude_hybrid, ACTIVITY_KERNEL_WIDTH)

Processing 1066528
Pre-mask:
	Sleeps 767 
	Wakes 179
Post-mask:
	Sleeps 618 
	Wakes 132 
	Wakes masked 41
Processing 1360686
Pre-mask:
	Sleeps 831 
	Wakes 93
Post-mask:
	Sleeps 823 
	Wakes 88 
	Wakes masked 0
Processing 1449548
Pre-mask:
	Sleeps 848 
	Wakes 104
Post-mask:
	Sleeps 744 
	Wakes 92 
	Wakes masked 7
Processing 1455390
Pre-mask:
	Sleeps 871 
	Wakes 84
Post-mask:
	Sleeps 870 
	Wakes 80 
	Wakes masked 0
Processing 1818471
Pre-mask:
	Sleeps 948 
	Wakes 10
Post-mask:
	Sleeps 948 
	Wakes 10 
	Wakes masked 0
Processing 2598705
Pre-mask:
	Sleeps 934 
	Wakes 20
Post-mask:
	Sleeps 740 
	Wakes 14 
	Wakes masked 6
Processing 2638030
Pre-mask:
	Sleeps 825 
	Wakes 123
Post-mask:
	Sleeps 825 
	Wakes 123 
	Wakes masked 0
Processing 3997827
Pre-mask:
	Sleeps 933 
	Wakes 25
Post-mask:
	Sleeps 929 
	Wakes 25 
	Wakes masked 0
Processing 4018081
Pre-mask:
	Sleeps 419 
	Wakes 79
Post-mask:
	Sleeps 419 
	Wakes 65 
	Wakes masked 8
Processing 4314139
Pre-mask:
	Sleeps 857 
	Wakes 104
Post-mask:
	Sl

In [12]:
# Prepare a bundle to save
save_preprocessing_to = walch_path.joinpath("walch_preprocessed_data.npy")
print(f"Saving to {save_preprocessing_to}...")

with open(save_preprocessing_to, 'wb') as f:
    np.save(f, preprocessed_data_walch)

save_preprocessing_to = hybrid_path.joinpath("hybrid_preprocessed_data.npy")
print(f"Saving to {save_preprocessing_to}...")
with open(save_preprocessing_to, 'wb') as f:
    np.save(f, preprocessed_data_hybrid)

Saving to /Users/eric/Engineering/Work/sleepers_analysis/notebooks/pre_processed_data/walch/walch_preprocessed_data.npy...
Saving to /Users/eric/Engineering/Work/sleepers_analysis/notebooks/pre_processed_data/hybrid/hybrid_preprocessed_data.npy...
